In [8]:
year = 2023
day = 24

In [9]:
%pip install sympy


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from aocd import submit
from aocd.models import Puzzle
from functools import reduce
import numpy as np

np.set_printoptions(edgeitems=30, linewidth=100000,
                    formatter=dict(float=lambda x: "%s" % x))

puzzle = Puzzle(year=year, day=day)
data = puzzle.input_data
left_bound = 200_000_000_000_000
right_bound = 400_000_000_000_000
# data = puzzle.examples[0].input_data
# left_bound = 7
# right_bound = 27

data = data.strip()
data = data.split("\n")
data = [list(map(int, d.replace("@", "").replace(",", "").split())) for d in data]
data[0]

[291493672529314, 259618209733833, 379287136024123, -9, 119, -272]

In [33]:
def solve_lines(left, right):
    a1, b1, c1, d1, = left[3], left[0], left[4], left[1]
    a2, b2, c2, d2, = right[3], right[0], right[4], right[1]
    try:
        t2 = (a1 * (d2 - d1) + c1 * (b1 - b2)) / (a2 * c1 - c2 * a1)
        t1 = (t2 * a2 + b2 - b1) / a1
    except ZeroDivisionError:
        # parallel lines
        return None, None, None, None

    xl = t1 * a1 + b1
    xb = t2 * a2 + b2
    yl = t1 * c1 + d1
    yb = t2 * c2 + d2

    return xl, yl, t1, t2

In [34]:
def check(l, r):
    x, y, t1, t2 = solve_lines(l, r)
    if not x:
        return False
    return (left_bound <= x <= right_bound) and (left_bound <= y <= right_bound) and (t1 >= 0) and (t2 >= 0)

In [35]:
from itertools import combinations

answer = sum([check(l, r) for l, r in combinations(data, 2)])
submit(answer, part="a", year=year, day=day)

Part a already solved with same answer: 17776


In [18]:
from sympy import solve, Symbol
from sympy.utilities.lambdify import lambdify

t1 = Symbol("t1")
t2 = Symbol("t2")
a1 = Symbol("a1")
b1 = Symbol("b1")
c1 = Symbol("c1")
d1 = Symbol("d1")
e1 = Symbol("e1")
f1 = Symbol("f1")
a2 = Symbol("a2")
b2 = Symbol("b2")
c2 = Symbol("c2")
d2 = Symbol("d2")
e2 = Symbol("e2")
f2 = Symbol("f2")

rock_vx = Symbol("rock_vx")
rock_vy = Symbol("rock_vy")
rock_x = Symbol("rock_x")
rock_y = Symbol("rock_y")
rock_vz = Symbol("rock_vz")
rock_z = Symbol("rock_z")

sol = solve([a1 * t1 + b1 - (rock_vx * t1 + rock_x),
             c1 * t1 + d1 - (rock_vy * t1 + rock_y),
             a2 * t2 + b2 - (rock_vx * t2 + rock_x),
             c2 * t2 + d2 - (rock_vy * t2 + rock_y)],
            [t1,
             t2,
             rock_y,
             rock_x])

compute_rock_y = lambdify([a1, b1, c1, d1, a2, b2, c2, d2, rock_vx, rock_vy], sol[rock_y])
compute_rock_x = lambdify([a1, b1, c1, d1, a2, b2, c2, d2, rock_vx, rock_vy], sol[rock_x])
compute_t1 = lambdify([a1, b1, c1, d1, a2, b2, c2, d2, rock_vx, rock_vy], sol[t1])
compute_t2 = lambdify([a1, b1, c1, d1, a2, b2, c2, d2, rock_vx, rock_vy], sol[t2])

In [19]:
sol2 = solve([a1 * t1 + b1 - (rock_vx * t1 + rock_x),
             c1 * t1 + d1 - (rock_vy * t1 + rock_y),
             e1 * t1 + f1 - (rock_vz * t1 + rock_z),
             a2 * t2 + d2 - (rock_vx * t2 + rock_x),
             c2 * t2 + d2 - (rock_vy * t2 + rock_y),
             e2 * t2 + f2 - (rock_vz * t2 + rock_z)
             ],
            [rock_vz, rock_z])
compute_rock_vz = lambdify([e1, f1, e2, f2, t1, t2], sol2[rock_vz])
compute_rock_z = lambdify([e1, f1, e2, f2, t1, t2], sol2[rock_z])

In [21]:
from itertools import product
SEARCH_RANGE = 300
for rock_vx, rock_vy in product(range(-SEARCH_RANGE, SEARCH_RANGE), range(-SEARCH_RANGE, SEARCH_RANGE)):
    rock_ys = []
    rock_xs = []
    N = 3
    for i in range(N):
        l, r = data[i], data[i + 1]
        a1, b1, c1, d1, e1, f1 = l[3], l[0], l[4], l[1], l[5], l[2]
        a2, b2, c2, d2, e2, f2 = r[3], r[0], r[4], r[1], r[5], r[2]

        try:
            rock_x = compute_rock_x(a1, b1, c1, d1, a2, b2, c2, d2, rock_vx, rock_vy)
            rock_y = compute_rock_y(a1, b1, c1, d1, a2, b2, c2, d2, rock_vx, rock_vy)
        except ZeroDivisionError:
            continue

        rock_ys.append(rock_y)
        rock_xs.append(rock_x)

    if np.ptp(rock_ys) < 0.001 and np.ptp(rock_xs) < 0.001 and len(rock_ys) == N:
        t1 = compute_t1(a1, b1, c1, d1, a2, b2, c2, d2, rock_vx, rock_vy)
        t2 = compute_t2(a1, b1, c1, d1, a2, b2, c2, d2, rock_vx, rock_vy)

        rock_vz = compute_rock_vz(e1, f1, e2, f2, t1, t2)
        rock_z = compute_rock_z(e1, f1, e2, f2, t1, t2)

        print(f"rock_vx: {rock_vx}, rock_vy: {rock_vy} rock_x: {int(rock_x)} rock_y: {int(rock_y)} rock_vz: {int(rock_vz)} rock_z: {int(rock_z)}")

        break
else:
    print(f"No solution found")

answer = int(rock_x + rock_y + rock_z)

rock_vx: -193, rock_vy: -230 rock_x: 370994826025810 rock_y: 410411158485339 rock_vz: 218 rock_z: 167572107691063


In [22]:
submit(answer, part="b", year=year, day=day)

Part b already solved with same answer: 948978092202212
